In [9]:
import requests, json, numpy as np # for requesting URLs, converting to JSON format
from datetime import datetime as dt
from datetime import date
from geopy.geocoders import Nominatim # for finding latitude, longitude of city
from bs4 import BeautifulSoup # for web scraping
import matplotlib.pyplot as plt # for plotting

city = str(input("Please enter 'City, State': ")) # asks user for city. City can be in form of CITY, STATE

geolocator = Nominatim(user_agent="weather_app") # these four lines takes the city or zip code and finds its latitude and longitude
location = geolocator.geocode(city) # creates a class of the location
latitude = location.latitude
longitude = location.longitude

current_date = str((date.today()).strftime("%m-%d-%Y")) # gives today's date in format MM-DD-YYYY

api_key = "0c981865ac3b99b146db363335c9cf90" # this is a sample API key from openweathermap.org

def unix_to_datetime(unix_time): # converts from unix time to standard date and time
                                 # unix time is the amount of seconds since the start of 1970

    standard_date = (dt.fromtimestamp(unix_time)).strftime('%m-%d-%Y %H:%M:%S') # puts in format MONTH-DAY-YEAR HOUR-MINUTE-SECOND

    return standard_date

def datetime_to_unix(year,month,day,hour,minute): # converts from standard date and time to unix time

    standard_date = dt(year,month,day,hour,minute) # converts date to a standard format

    unix_time = int((standard_date - dt(1970, 1, 1)).total_seconds()) # finds total seconds since start of 1970

    return unix_time

def current(latitude, longitude, data): # Gives data about weather at this very moment

    current_url = f"http://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={api_key}&units=imperial" # link returns JSON file of the current weather data

    response = requests.get(current_url) # requests the website data

    response_text = response.text # converts website data into text

    current_json = json.loads(response_text) # converts text into json format

    if data == "temp": # current temperature
        return current_json["main"]["temp"]
    elif data == "temp_low": # low margin of error for current temp
        return current_json["main"]["temp_min"] 
    elif data == "temp_high": # high margin of error for current temp
        return current_json["main"]["temp_max"] 
    elif data == "feels_like": # feels-like temp
        return current_json["main"]["feels_like"]
    elif data == "weather": # weather
        return current_json["weather"][0]["description"]
    elif data == "humidity": # humidity
        return current_json["main"]["humidity"]
    elif data == "wind_speed": # wind speed
        return current_json["wind"]["speed"]
    elif data == "wind_direction": # wind direction, converting from degrees to cardinal directions
        if current_json["wind"]["deg"] < 90:
            return "Northeast"
        elif 90 <= current_json["wind"]["deg"] < 180:
            return "Southeast"
        elif 180 <= current_json["wind"]["deg"] < 270:
            return "Southwest"
        elif 270 <= current_json["wind"]["deg"] < 360:
            return "Northwest"

def forecast(latitude, longitude, data, days): # Gives weekly forecast and today's weather

    forecast_url = f"https://api.openweathermap.org/data/2.5/onecall?lat={latitude}&lon={longitude}&exclude=minutely,hourly,current&appid={api_key}&units=imperial" # link returns JSON file of future weather data

    response = requests.get(forecast_url)

    response_text = response.text

    forecast_json = json.loads(response_text)

    def weekly_high(): # gives the highest temp for the next week

        highs = []

        for n in range(1,8):
            highs.append(forecast_json["daily"][n]["temp"]["max"])
        
        weekly_high = max(highs)

        return weekly_high

    def weekly_low(): # gives the lowest temp for the next week

        lows = []

        for n in range(1,8):
            lows.append(forecast_json["daily"][n]["temp"]["min"])
        
        weekly_low = min(lows)

        return weekly_low

    def weekly_average(): # gives average temperature for the next week
        
        temps = []

        for n in range(1,8):
            temps.append(forecast_json["daily"][n]["temp"]["day"])
        
        return np.round(np.average(temps), 2) # finds average of temps and rounds it to two decimal places

    if data == "temp":
        return forecast_json["daily"][days]["temp"]["day"]
    elif data == "temp_low": # day low temp
        return forecast_json["daily"][days]["temp"]["min"]
    elif data == "temp_high": # day high temp
        return forecast_json["daily"][days]["temp"]["max"]
    elif data == "temp_average": # averages low and high temps
        return np.round(((forecast_json["daily"][days]["temp"]["min"] + forecast_json["daily"][days]["temp"]["max"]) / 2), 2) 
    elif data == "weekly_high":
        return weekly_high()
    elif data == "weekly_low":
        return weekly_low()
    elif data == "weekly_average":
        return weekly_average()
    elif data == "feels_like":
        return forecast_json["daily"][days]["feels_like"]["day"]
    elif data == "weather":
        return forecast_json["daily"][days]["weather"][0]["description"]
    elif data == "humidity":
        return forecast_json["daily"][days]["humidity"]
    elif data == "wind_speed":
        return forecast_json["daily"][days]["wind_speed"]
    elif data == "wind_direction":
        if forecast_json["daily"][days]["wind_deg"] < 90:
            return "Northeast"
        elif 90 <= forecast_json["daily"][days]["wind_deg"] < 180:
            return "Southeast"
        elif 180 <= forecast_json["daily"][days]["wind_deg"] < 270:
            return "Southwest"
        elif 270 <= forecast_json["daily"][days]["wind_deg"] <= 360:
            return "Northwest"
    elif data == "sunrise": # sunrise time
        return unix_to_datetime(forecast_json["daily"][days]["sunrise"])
    elif data == "sunset": # sunset time
        return unix_to_datetime(forecast_json["daily"][days]["sunset"])
    elif data == "rain": # rainfall in mm
        try:
            return forecast_json["daily"][days]["rain"] 
        except KeyError: # if rain doesn't exist in JSON, then there will be no rain
            return 0
    elif data == "snow": # snowfall in mm
        try:
            return forecast_json["daily"][days]["snow"] 
        except KeyError: # if snow doesn't exist in JSON, then there will be no snow
            return 0

def history(latitude, longitude, data, date): # Gives history up to 5 days back. Data is from 1 PM EST on those days

    dateSplitUp = date.split("-")
    month = int(dateSplitUp[0])
    day = int(dateSplitUp[1])
    year = int(dateSplitUp[2])
    hour = 1
    minute = 0

    unix_time = datetime_to_unix(year,month,day,hour,minute) # converts given datetime to unix time

    history_url = f"https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={unix_time}&appid={api_key}&units=imperial" # link returns JSON file of historical weather data

    response = requests.get(history_url)

    response_text = response.text

    history_json = json.loads(response_text)

    if data == "temp":
        return history_json["current"]["temp"]
    elif data == "humidity":
        return history_json["current"]["humidity"]
    elif data == "weather":
        return history_json["current"]["weather"][0]["description"]
    elif data == "wind_speed":
        return history_json["current"]["wind_speed"]
    elif data == "wind_direction":
        if history_json["current"]["wind_deg"] < 90:
            return "Northeast"
        elif 90 <= history_json["current"]["wind_deg"] < 180:
            return "Southeast"
        elif 180 <= history_json["current"]["wind_deg"] < 270:
            return "Southwest"
        elif 270 <= history_json["current"]["wind_deg"] <= 360:
            return "Northwest"
    elif data == "sunrise":
        return unix_to_datetime(history_json["current"]["sunrise"])
    elif data == "sunset":
        return unix_to_datetime(history_json["current"]["sunset"])

def on_this_day(city,data,date): # returns data about the temperature on a certain day since 1970

    dateSplitUp = date.split("-") # these 3 lines split the month and day into integers
    month = int(dateSplitUp[0])
    day = int(dateSplitUp[1])

    year_range = range(1970,current_year) # the website tracks from 1945, but doing since 1970 will shorten the execution time

    temp_average_list = []
    temp_min_list = []
    temp_max_list = []

    try:
        citySplitUp = city.split(", ") # for if user put space after comma
        city = citySplitUp[0]
        state = citySplitUp[1]

    except IndexError:
        citySplitUp = city.split(",") # for if user didn't put space after comma
        city = citySplitUp[0]
        state = citySplitUp[1]

    for year in year_range: # finds the link for years 1970 to 2021 and returns the data requested

        on_this_day_url = f"https://www.almanac.com/weather/history/{state}/{city}/{year}-{month}-{day}" # link returns HTML data
        
        response = requests.get(on_this_day_url)

        response_content = response.content

        html = BeautifulSoup(response_content, 'html.parser') # allows data to be parsed as HTML

        temp_average = html.find(class_ = "weatherhistory_results_datavalue temp").find(class_ = "value").get_text() # HTML location for average

        temp_min = html.find(class_ = "weatherhistory_results_datavalue temp_mn").find(class_ = "value").get_text() # HTML location for minimum

        temp_max = html.find(class_ = "weatherhistory_results_datavalue temp_mx").find(class_ = "value").get_text() # HTML location for maximum

        temp_units = (html.find(class_ = "weatherhistory_results_datavalue temp_mn").find(class_ = "units").get_text())[1] # HTML location for units

        if data == "average":
            if temp_units == "C": # if that year was recorded in celsius, convert it to fahrenheit before adding to the list
                temp_average_list.append((9/5)*float(temp_average) + 32)
            else:
                temp_average_list.append(float(temp_average))

        elif data == "min":
            if temp_units == "C":
                temp_min_list.append((9/5)*float(temp_min) + 32)
            else:
                temp_min_list.append(float(temp_min))

        elif data == "max":
            if temp_units == "C":
                temp_max_list.append((9/5)*float(temp_max) + 32)
            else:
                temp_max_list.append(float(temp_max))

    if data == "average": # creates a plot of all the data from 1970 to 2021 
        fig, av_plt = plt.subplots(figsize = (10, 5))
        av_plt.plot(year_range, temp_average_list)
        av_plt.scatter(year_range, temp_average_list)
        av_plt.set_title(f"Average temperatures in {city}, {state} on {date} from 1970 to 2021", fontsize = 18)
        av_plt.set_xlabel("Year", fontsize = 18)
        av_plt.set_ylabel("Temperature (°F)", fontsize = 18)
        return np.round(np.average(temp_average_list), 2)

    if data == "min":
        fig, min_plt = plt.subplots(figsize = (10, 5))
        min_plt.plot(year_range, temp_min_list)
        min_plt.scatter(year_range, temp_min_list)
        min_plt.set_title(f"Minimum temperatures in {city}, {state} on {date} from 1970 to 2021", fontsize = 18)
        min_plt.set_xlabel("Year", fontsize = 18)
        min_plt.set_ylabel("Temperature (°F)", fontsize = 18)
        return min(temp_min_list)

    if data == "max":
        fig, max_plt = plt.subplots(figsize = (10, 5))
        max_plt.plot(year_range, temp_max_list)
        max_plt.scatter(year_range, temp_max_list)
        max_plt.set_title(f"Maximum temperatures in {city}, {state} on {date} from 1970 to 2021", fontsize = 18)
        max_plt.set_xlabel("Year", fontsize = 18)
        max_plt.set_ylabel("Temperature (°F)", fontsize = 18)
        return max(temp_max_list)

def compare_today(): # compares today's temperature to today's historic average

    today_average = forecast(latitude,longitude,"temp_average",0) # average temp for today
    all_time_average = on_this_day(city,"average",current_date) # all-time average for this day
    plt.close() # gets rid of the plot returned from the on_this_day function
    if all_time_average <= today_average:

        return f"Today's temperature of {today_average}°F is hotter than the historic average of {all_time_average}°F"

    else:

        return f"Today's temperature of {today_average}°F is colder than the historic average of {all_time_average}°F"

Please enter 'City, State': gainesville, fl


'light rain'